In [479]:
import os
import re
import pandas as pd
from tqdm import tqdm
from itertools import compress

In [303]:
# get all subdirectories in data folder
data_path = r'../data'
dataset_labels = []
for dir in os.scandir(data_path):
    dataset_labels.append(dir.name)

In [304]:
# get all log files for each data set
dataset_logs = []
log_pattern = r'[A-Za-z0-9]*.log'
for i, dir_list in enumerate([os.listdir(data_path + '/' + dir) for dir in dataset_labels]):
    specific_log_title = []
    specific_log = [] 
    for file in dir_list:
        if re.search(log_pattern, file):
            specific_log_title.append(file)
            f = open(data_path + '/' + dataset_labels[i] + '/' + file)
            specific_log.append(f.read())
            f.close()
    dataset_logs.append((specific_log_title,specific_log))



In [305]:
log = dataset_logs[0][1]
split_logs = str(log).split(r'end of world_state')
split_logs = split_logs[:-1]
len(split_logs)

20

In [306]:
for i, log in enumerate(split_logs):
    split_logs[i] = log.split('end of events')[0]
    
flight_envelopes = [] # contains all timestep data
events = [] # contains the events at the end of each log file
for i, log in enumerate(split_logs):
    flight_envelopes.append(log.split('events')[0])
    events.append(log.split('events')[1])

In [307]:

df_arrays = []
for i in flight_envelopes[0].split(r'\n'):
    df_arrays.append(i.split(','))
df = pd.DataFrame(df_arrays)


In [308]:
df.iloc[0,:]

0                 ['Player: t: 0.00
1                   MSLAlt: 1279.81
2                speed(m/s): 240.83
3                     heading: 5.50
4                     x: -302927.78
5                        y: 1279.81
6                      z: 678108.13
7                       pitch: 1.74
8                       bank: -0.00
9     yaw: 314.88 Emitters.Mode: 0 
Name: 0, dtype: object

In [309]:
df.columns = [re.search(r'([A-Za-z0-9(/)]*):', s).group(0) for s in df.iloc[0,:]]

In [310]:
sub_pattern = r'["[\' \sA-Za-z0-9(/)]*:'
re.sub(sub_pattern, '', df.iloc[0,0])

' 0.00'

In [311]:
df.iloc[0,0]

"['Player: t: 0.00"

In [312]:
sam_df_list = []
def pop_row(row) -> pd.DataFrame:
    if row['Player:'].startswith('Sam'):
         df.drop(index=row.name,inplace=True)
         sam_row = row[0]
         sam_df_list.append([sam_row])
    else:
        pass
    return df
df.apply(pop_row, axis=1)
sam_df = pd.DataFrame(sam_df_list)

#df = df[df['x:'].notnull()]

In [313]:
df = df[df['x:'].notnull()]
print(f'Sam rows :{len(sam_df_list)} + player rows: {len(df.index)} = {len(sam_df_list) + len(df.index)}')

Sam rows :1163 + player rows: 5104 = 6267


In [314]:
df.iloc[2801,0]

'Player: t: 43.52'

In [315]:
df.applymap(lambda x: re.sub(sub_pattern, '', x))

,Player:,MSLAlt:,speed(m/s):,heading:,x:,y:,z:,pitch:,bank:,yaw:
0,0.00,1279.81,240.83,5.50,-302927.78,1279.81,678108.13,1.74,-0.00,314.88 Emitters. 0
1,0.02,1279.81,240.83,5.50,-302924.88,1279.81,678105.21,1.74,0.00,314.88 Emitters. 0
2,0.04,1279.81,240.83,5.50,-302921.44,1279.81,678101.75,1.73,0.00,314.88 Emitters. 0
3,0.06,1279.81,236.72,5.50,-302916.37,1279.81,678096.65,1.65,0.01,314.88 Emitters. 0
4,0.11,1279.80,236.61,5.50,-302907.86,1279.80,678088.10,1.49,0.03,314.87 Emitters. 0
...,...,...,...,...,...,...,...,...,...,...
6262,79.24,1850.54,365.82,3.23,-313589.53,1850.64,669901.21,3.81,-0.23,185.14 Emitters.1.Type. -2. 0. 130. 0
6263,79.26,1850.90,365.82,3.23,-313595.47,1850.94,669900.62,3.81,-0.24,185.14 Emitters.1.Type. -2. 0. 130. 0
6264,79.27,1851.14,365.83,3.23,-313601.41,1851.24,669900.02,3.81,-0.24,185.14 Emitters.1.Type. -2. 0. 130. 0
6265,79.29,1851.50,365.83,3.23,-313607.74,1851.56,669899.39,3.81,-0.24,185.14 Emitters.1.Type. -2. 0. 130. 0


In [316]:
def 

SyntaxError: invalid syntax (1885169875.py, line 1)

In [319]:
sub_pattern = r'["[\' \sA-Za-z0-9(/)]*:'
string = "Player: t: 0.14, MSLAlt: 1279.79, speed(m/s): 236.52, heading: 5.50, x: -302902.43, y: 1279.79, z: 678082.64, pitch: 1.41, bank: 0.04, yaw: 314.87 Emitters.Mode: 0 "
for i in string.split(','):
    print(re.sub(sub_pattern,'',i))

 0.14
 1279.79
 236.52
 5.50
 -302902.43
 1279.79
 678082.64
 1.41
 0.04
 314.87 Emitters. 0 


In [411]:
import re
import os
from typing import Literal
import pandas as pd
from tqdm import tqdm
import numpy as np
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
def read_logs(data_path) -> tuple[list[str], list[str]]:  
    # get all subdirectories in data folder
    dataset_labels = []
    for dir in os.scandir(data_path):
        dataset_labels.append(dir.name)

    # get all log files for each data set
    dataset_log_titles = []
    log_pattern = r'[A-Za-z0-9]*.log'
    for i, dir_list in enumerate([os.listdir(data_path + '/' + dir) for dir in dataset_labels]):
        specific_log_title = []
        for file in dir_list:
            if re.search(log_pattern, file):
                specific_log_title.append(file)
                #f = open(data_path + '/' + dataset_labels[i] + '/' + file)
                #specific_log.append(f.read())
                #f.close()
        dataset_log_titles.append(specific_log_title)
    print(f'Successfully found the following labels : {dataset_labels}')
    for i, label in enumerate(dataset_labels):
        print(f'Found {len(dataset_log_titles[i][0])} logs for the "{label}" label')
    

    return dataset_labels, dataset_log_titles

def read_log_file_contents(folder, filename, file_path) -> tuple[pd.DataFrame, pd.DataFrame]:
    # new dataframe
    df = pd.DataFrame(columns=['CATEGORY', 'RUN_ID', 'TIME', 'ALT', 'SPEED', 'PITCH', 'BANK', 'RWR', 'GT_MISSILE_DIST', 'T_UNTIL_IMPACT'])
    event_df = pd.DataFrame(columns=['CATEGORY','RUN_ID','IMPACT_TIME'])
    # open the file
    with open(file_path, 'r') as f:
        # if line starts with "Player", then parse the line
        text = f.readlines()
        f.close()
    
    for i, line in enumerate(text):
        if line.startswith('Player'):
            # new dict for each line
            line_dict = {}

            # category is the folder name
            line_dict['CATEGORY'] = folder

            # run_id is the filename
            line_dict['RUN_ID'] = filename

            sub_pattern = r'["[\' \sA-Za-z0-9(/)]*:'
            # time follows "t: "
            
            text_obj = re.sub(sub_pattern,'',line)
            text_obj = text_obj.split(',')
            line_dict['TIME'] = text_obj[0]

            # altitude follows "MSLAlt: "
            line_dict['ALT'] = text_obj[1]

            # speed follows "speed(m/s): "
            line_dict['SPEED'] = text_obj[2]

            # pitch follows "pitch: "
            line_dict['PITCH'] = text_obj[7]

            # bank follows "bank: "
            line_dict['BANK'] = text_obj[8]

            # RWR is everything after "Emitters."
            line_dict['RWR'] = line.split('Emitters.')[1]
            
            # placeholder to be backfilled later based on detected
            line_dict['T_UNTIL_IMPACT'] = np.nan
            
            # if nextline doesn't start with "sam", GT_MISSILE_DIST is infinite
            #nextline = text[i+1]
            if i + 1 <= len(text)-1:
                if not text[i+1].startswith('Sam'):
                    # GT_MISSILE_DIST is infinite
                    line_dict['GT_MISSILE_DIST'] = np.inf

                else:
                    # Calculate GT_MISSILE_DIST from X, Y Z of aircraft and missile
                    # x, y, z of aircraft
                    ac_X = float(line.split('x: ')[1].split(',')[0])
                    ac_Y = float(line.split('y: ')[1].split(',')[0])
                    ac_Z = float(line.split('z: ')[1].split(',')[0])

                    # X, Y, Z of missile
                    sam_X = float(text[i+1].split('x: ')[1].split(' ')[0])
                    sam_Y = float(text[i+1].split('y: ')[1].split(' ')[0])
                    sam_Z = float(text[i+1].split('z: ')[1].split(' ')[0])

                    # calculate distance
                    line_dict['GT_MISSILE_DIST'] = np.sqrt((ac_X - sam_X)**2 + (ac_Y - sam_Y)**2 + (ac_Z - sam_Z)**2)
        
            # append the line to the dataframe
            df = df.append(line_dict, ignore_index=True)
        elif line.startswith('\t\ttype\t=\t"hit"'):
            
            event_dict = {}
            
            # category is the folder name
            event_dict['CATEGORY'] = folder

            # run_id is the filename
            event_dict['RUN_ID'] = filename
            
            # impact_time is the t = value in the debrief section of the log file, it comes 3 rows after the 'hit' message
            time_row = text[i+3]
            event_dict['IMPACT_TIME'] = float(re.sub(r'[\st=,]*', '', time_row))
            event_df = event_df.append(event_dict, ignore_index=True)

    #return df
    return (df, event_df)



In [375]:
dataset_labels

['180_away', '180_towards', 'baseline', 'climb', 'roll_and_dive']

In [467]:
#from clean import read_logs, read_log_file_contents
from tqdm import tqdm


data_path = r'../data'
dataset_labels, dataset_log_titles = read_logs(data_path=data_path) # data is tuple(maneuver, tuple(log_filename, log_contents))

dataset_dfs = []
dataset_event_dfs = []
for i, l in enumerate(dataset_labels[:1]):
    dfs = []
    event_dfs = []
    print(l)
    for t in tqdm(dataset_log_titles[i]):
        df, event_df = read_log_file_contents(l,t,data_path+ '/' + l + '/' + t)
        dfs.append(df)
        event_dfs.append(event_df)

    dataset_dfs.append(dfs)
    dataset_event_dfs.append(event_dfs)



Successfully found the following labels : ['180_away', '180_towards', 'baseline', 'climb', 'roll_and_dive']
Found 20 logs for the "180_away" label
Found 20 logs for the "180_towards" label
Found 20 logs for the "baseline" label
Found 20 logs for the "climb" label
Found 20 logs for the "roll_and_dive" label
180_away


100%|██████████| 20/20 [08:01<00:00, 24.10s/it]


In [497]:

def drop_clean_runs(dataset_dfs, dataset_event_dfs):
    mask = np.invert(np.array([event_df.empty for event_df in dataset_event_dfs[0]]))
    dataset_dfs = list(compress(dataset_dfs[0],mask))
    dataset_event_dfs = list(compress(dataset_event_dfs[0],mask))
    return dataset_dfs, dataset_event_dfs

#dataset_dfs, dataset_event_dfs = drop_clean_runs(dataset_dfs, dataset_event_dfs)

print(dataset_dfs[0])
print(dataset_event_dfs[0])



      CATEGORY                RUN_ID    TIME       ALT    SPEED  PITCH  \
0     180_away  Export1662073759.log    0.00   1827.03   234.36   1.84   
1     180_away  Export1662073759.log    0.01   1827.03   234.36   1.84   
2     180_away  Export1662073759.log    0.03   1827.03   234.36   1.82   
3     180_away  Export1662073759.log    0.06   1827.03   234.32   1.76   
4     180_away  Export1662073759.log    0.09   1827.03   237.44   1.63   
...        ...                   ...     ...       ...      ...    ...   
2805  180_away  Export1662073759.log   43.55   1728.94   291.94   2.69   
2806  180_away  Export1662073759.log   43.56   1728.57   291.94   3.32   
2807  180_away  Export1662073759.log   43.58   1728.20   291.94   3.84   
2808  180_away  Export1662073759.log   43.59   1727.96   291.93   4.12   
2809  180_away  Export1662073759.log   43.61   1727.72   291.94   4.35   

         BANK         RWR  GT_MISSILE_DIST  T_UNTIL_IMPACT  
0       -0.00  Mode: 0 \n              inf        

In [517]:
def t_until_impact(row, impact):
    row['T_UNTIL_IMPACT'] = impact - float(row['TIME'])
    return row
impact = dataset_event_dfs[0].iloc[0,2]
dataset_dfs[0].apply(t_until_impact,axis=1, impact=impact)

,CATEGORY,RUN_ID,TIME,ALT,SPEED,PITCH,BANK,RWR,GT_MISSILE_DIST,T_UNTIL_IMPACT
0,180_away,Export1662073759.log,0.00,1827.03,234.36,1.84,-0.00,Mode: 0 \n,inf,42.642
1,180_away,Export1662073759.log,0.01,1827.03,234.36,1.84,0.00,Mode: 0 \n,inf,42.632
2,180_away,Export1662073759.log,0.03,1827.03,234.36,1.82,0.01,Mode: 0 \n,inf,42.612
3,180_away,Export1662073759.log,0.06,1827.03,234.32,1.76,0.07,Mode: 0 \n,inf,42.582
4,180_away,Export1662073759.log,0.09,1827.03,237.44,1.63,0.21,Mode: 0 \n,inf,42.552
...,...,...,...,...,...,...,...,...,...,...
2805,180_away,Export1662073759.log,43.55,1728.94,291.94,2.69,-19.57,Mode: 0 \n,1990.409532,-0.908
2806,180_away,Export1662073759.log,43.56,1728.57,291.94,3.32,-11.22,Mode: 0 \n,1983.535184,-0.918
2807,180_away,Export1662073759.log,43.58,1728.20,291.94,3.84,-2.91,Mode: 0 \n,1975.709080,-0.938
2808,180_away,Export1662073759.log,43.59,1727.96,291.93,4.12,2.60,Mode: 0 \n,1969.249551,-0.948


In [516]:
for i, df in enumerate(dataset_dfs):
    impact = dataset_event_dfs[i].iloc[0,2]
    dataset_dfs[i] = dataset_dfs[i].apply(t_until_impact, axis=1, impact=impact)

print(dataset_dfs[2]['T_UNTIL_IMPACT'])

0       42.895
1       42.875
2       42.855
3       42.835
4       42.785
         ...  
2799    -0.655
2800    -0.665
2801    -0.675
2802    -0.695
2803    -0.705
Name: T_UNTIL_IMPACT, Length: 2804, dtype: float64


In [551]:
df = dataset_dfs[0]
df

,CATEGORY,RUN_ID,TIME,ALT,SPEED,PITCH,BANK,RWR,GT_MISSILE_DIST,T_UNTIL_IMPACT
0,180_away,Export1662073759.log,0.00,1827.03,234.36,1.84,-0.00,Mode: 0 \n,inf,42.642
1,180_away,Export1662073759.log,0.01,1827.03,234.36,1.84,0.00,Mode: 0 \n,inf,42.632
2,180_away,Export1662073759.log,0.03,1827.03,234.36,1.82,0.01,Mode: 0 \n,inf,42.612
3,180_away,Export1662073759.log,0.06,1827.03,234.32,1.76,0.07,Mode: 0 \n,inf,42.582
4,180_away,Export1662073759.log,0.09,1827.03,237.44,1.63,0.21,Mode: 0 \n,inf,42.552
...,...,...,...,...,...,...,...,...,...,...
2805,180_away,Export1662073759.log,43.55,1728.94,291.94,2.69,-19.57,Mode: 0 \n,1990.409532,-0.908
2806,180_away,Export1662073759.log,43.56,1728.57,291.94,3.32,-11.22,Mode: 0 \n,1983.535184,-0.918
2807,180_away,Export1662073759.log,43.58,1728.20,291.94,3.84,-2.91,Mode: 0 \n,1975.709080,-0.938
2808,180_away,Export1662073759.log,43.59,1727.96,291.93,4.12,2.60,Mode: 0 \n,1969.249551,-0.948


In [552]:
def drop_runs_after_hit(df):
    df = df[df['T_UNTIL_IMPACT'] > 0]
    return df
drop_runs_after_hit(df)

,CATEGORY,RUN_ID,TIME,ALT,SPEED,PITCH,BANK,RWR,GT_MISSILE_DIST,T_UNTIL_IMPACT
0,180_away,Export1662073759.log,0.00,1827.03,234.36,1.84,-0.00,Mode: 0 \n,inf,42.642
1,180_away,Export1662073759.log,0.01,1827.03,234.36,1.84,0.00,Mode: 0 \n,inf,42.632
2,180_away,Export1662073759.log,0.03,1827.03,234.36,1.82,0.01,Mode: 0 \n,inf,42.612
3,180_away,Export1662073759.log,0.06,1827.03,234.32,1.76,0.07,Mode: 0 \n,inf,42.582
4,180_away,Export1662073759.log,0.09,1827.03,237.44,1.63,0.21,Mode: 0 \n,inf,42.552
...,...,...,...,...,...,...,...,...,...,...
2742,180_away,Export1662073759.log,42.58,1733.57,292.88,1.45,82.20,Mode: 0 \n,48.053267,0.062
2743,180_away,Export1662073759.log,42.59,1733.63,292.95,1.45,82.22,Mode: 0 \n,41.018617,0.052
2744,180_away,Export1662073759.log,42.61,1733.71,293.00,1.44,82.25,Mode: 0 \n,33.667325,0.032
2745,180_away,Export1662073759.log,42.62,1733.77,293.05,1.44,82.27,Mode: 0 \n,26.570830,0.022


In [553]:
def find_last_RWR_lock(col) -> int:
    indices = []
    for i, r in enumerate(col):
        search = re.search(r'.*(missile_radio_guided Mode:)', r)
        if search is not None:
            indices.append(i)
    first_lock_index = int(indices[0])
    return first_lock_index  

SyntaxError: invalid syntax (2272330867.py, line 3)

In [554]:
df['RWR'].value_counts()

Mode: 0 \n                                                                                                                                                                                899
1.Type.level3: 102 level1: 2 level4: 23 level2: 16 Azimuth: -0.093582153320313 Power: 0.86565172672272 ID: 16777472 Priority: 230.86427307129 SignalType: lock Mode: 0 \n                   2
1.Type.level3: 102 level1: 2 level4: 23 level2: 16 Azimuth: -0.20732255280018 Power: 0.92439019680023 ID: 16777472 Priority: 230.92306518555 SignalType: lock Mode: 0 \n                    2
1.Type.level3: 102 level1: 2 level4: 23 level2: 16 Azimuth: -0.093297332525253 Power: 0.86537951231003 ID: 16777472 Priority: 230.86427307129 SignalType: lock Mode: 0 \n                   2
1.Type.level3: 102 level1: 2 level4: 23 level2: 16 Azimuth: -0.11911035329103 Power: 0.88688778877258 ID: 16777472 Priority: 230.88487243652 SignalType: lock Mode: 0 \n                    2
                                                  

In [594]:
indices = []
for i, r in enumerate(col):
    search = re.search(r'.*(missile_radio_guided Mode:)', r)
    if search is not None:
        indices.append(i)
first_index = indices[0]

In [603]:


def time_since_lock_on(row, first_lock_t):
    row['TIME_SINCE_LOCK'] = float(row['TIME']) - float(first_lock_t)
    return row
df =df.apply(time_since_lock_on, axis=1, first_lock_t=df.iloc[first_index, 2])

In [597]:
df.iloc[first_index, 2]

' 33.51'

In [604]:
def drop_early_and_late_runs(df):
    df = df[df['T_UNTIL_IMPACT'] > 0]
    df = df[df['TIME_SINCE_LOCK'] > 0]
    return df
drop_early_and_late_runs(df)

,CATEGORY,RUN_ID,TIME,ALT,SPEED,PITCH,BANK,RWR,GT_MISSILE_DIST,T_UNTIL_IMPACT,TIME_SINCE_LOCK
2159,180_away,Export1662073759.log,33.52,1737.06,262.24,1.62,-1.41,1.Type.level3: 102 level1: 2 level4: 23 level2...,5745.003056,9.122,0.01
2160,180_away,Export1662073759.log,33.54,1737.06,262.24,1.62,-1.41,1.Type.level3: 102 level1: 2 level4: 23 level2...,5740.922495,9.102,0.03
2161,180_away,Export1662073759.log,33.55,1737.07,262.26,1.62,-1.41,1.Type.level3: 102 level1: 2 level4: 23 level2...,5736.533515,9.092,0.04
2162,180_away,Export1662073759.log,33.57,1737.08,262.27,1.62,-1.41,1.Type.level3: 102 level1: 2 level4: 23 level2...,5732.644254,9.072,0.06
2163,180_away,Export1662073759.log,33.58,1737.09,262.29,1.62,-1.41,1.Type.level3: 102 level1: 2 level4: 23 level2...,5728.163798,9.062,0.07
...,...,...,...,...,...,...,...,...,...,...,...
2742,180_away,Export1662073759.log,42.58,1733.57,292.88,1.45,82.20,Mode: 0 \n,48.053267,0.062,9.07
2743,180_away,Export1662073759.log,42.59,1733.63,292.95,1.45,82.22,Mode: 0 \n,41.018617,0.052,9.08
2744,180_away,Export1662073759.log,42.61,1733.71,293.00,1.44,82.25,Mode: 0 \n,33.667325,0.032,9.10
2745,180_away,Export1662073759.log,42.62,1733.77,293.05,1.44,82.27,Mode: 0 \n,26.570830,0.022,9.11
